In [3]:
import numpy as np
import zarr, dask
import netCDF4
import xarray as xr
import s3fs

In [4]:
ds1 = xr.open_dataset('gfdl-esm4_r1i1p1f1_w5e5_ssp119_pr_global_daily_2081_2090.nc')

In [5]:
ds2 = xr.open_dataset('gfdl-esm4_r1i1p1f1_w5e5_ssp119_pr_global_daily_2091_2100.nc')

In [6]:
%%time
ds3 = xr.concat([ds1, ds2], dim=['time', 'time'])

CPU times: total: 1min 2s
Wall time: 5min 15s


In [21]:
ds3

<xarray.Dataset>
Dimensions:     (lon: 720, lat: 360, time: 7304, concat_dim: 2)
Coordinates:
  * lon         (lon) float64 -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
  * lat         (lat) float64 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75
  * time        (time) datetime64[ns] 2081-01-01 2081-01-02 ... 2100-12-31
  * concat_dim  (concat_dim) object 'time' 'time'
Data variables:
    pr          (concat_dim, time, lat, lon) float32 dask.array<chunksize=(2, 7304, 47, 47), meta=np.ndarray>
Attributes:
    institution:  Potsdam Institute for Climate Impact Research (PIK)
    contact:      ISIMIP cross-sectoral science team <info@isimip.org> <https...
    references:   Lange (2019) <https://doi.org/10.5194/gmd-12-3055-2019> and...
    title:        ISIMIP3b bias-adjusted climate input data
    project:      Inter-Sectoral Impact Model Intercomparison Project phase 3...
    summary:      CMIP6 daily output data bias-adjusted and statistically dow...
    isimip_id:    ffedd285-b65b-4a01-9dec-6bf71cc7c7a7

In [8]:
ds3 = ds3.chunk({'time': -1, 'lat': 'auto', 'lon': 'auto'})
ds3

<xarray.Dataset>
Dimensions:     (lon: 720, lat: 360, time: 7304, concat_dim: 2)
Coordinates:
  * lon         (lon) float64 -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
  * lat         (lat) float64 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75
  * time        (time) datetime64[ns] 2081-01-01 2081-01-02 ... 2100-12-31
  * concat_dim  (concat_dim) object 'time' 'time'
Data variables:
    pr          (concat_dim, time, lat, lon) float32 dask.array<chunksize=(2, 7304, 47, 47), meta=np.ndarray>
Attributes:
    institution:  Potsdam Institute for Climate Impact Research (PIK)
    contact:      ISIMIP cross-sectoral science team <info@isimip.org> <https...
    references:   Lange (2019) <https://doi.org/10.5194/gmd-12-3055-2019> and...
    title:        ISIMIP3b bias-adjusted climate input data
    project:      Inter-Sectoral Impact Model Intercomparison Project phase 3...
    summary:      CMIP6 daily output data bias-adjusted and statistically dow...
    isimip_id:    ffedd285-b65b-4a01-9dec-6bf71cc7c7a7

In [9]:
del(ds1)
del(ds2)

In [32]:
s3_path = 's3://cities-indicators/bloomberg/pr_gfdl-esm4_2081-2100_ssp119_1.zarr'

In [33]:
s3 = s3fs.S3FileSystem()
store = s3fs.S3Map(root=s3_path, s3=s3, check=False)
compressor = zarr.Blosc(cname='zstd', clevel=3)
encoding = {vname: {'compressor': compressor} for vname in ds3.data_vars}

In [34]:
ds3.to_zarr(store=store, encoding=encoding, consolidated=True, mode='w')

PermissionError: The AWS Access Key Id you provided does not exist in our records.

In [ ]:
# Here I just gave up on sending it straight to S3.
# I saved it to my local machine and used CyberDuck to tranfer it to S3.

In [35]:
def s3open_cmip(path):
    fs = s3fs.S3FileSystem(anon=False)
    return s3fs.S3Map(path, s3=fs)
thefile = s3open_cmip(s3_path)
ds = xr.open_mfdataset([thefile], engine='zarr', parallel=True)

PermissionError: Forbidden